In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

In [2]:
data = pd.read_csv("ecg_autoencoder_dataset.csv")
scaler = StandardScaler()
data_normal = scaler.fit_transform(data)
X = data.iloc[:,:-1].values
y = data.iloc[:,-1]

In [3]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [4]:
x_train_normal = x_train[y_train==0]

In [5]:
input_layer = Input(shape=(x_train_normal.shape[-1],))
encoder = Dense(128,activation="relu")(input_layer)
encoder = Dense(64,activation="relu")(encoder)
latent = Dense(14,activation="relu")(encoder)

In [6]:
decoder = Dense(64,activation="relu")(latent)
decoder = Dense(128,activation="relu")(decoder)
output_layer = Dense(x_train_normal.shape[-1],activation="linear")(decoder)

In [7]:
autoencoder = Model(input_layer,output_layer)

In [8]:
autoencoder.compile(optimizer=Adam(learning_rate=0.001),loss="mse",metrics=["accuracy"])

In [9]:
autoencoder.fit(x_train_normal,x_train_normal,epochs=20,validation_split=0.2,batch_size=256,shuffle=True)

Epoch 1/20


6/6 [==============================] - 2s 81ms/step - loss: 0.9236 - accuracy: 0.0037 - val_loss: 0.7714 - val_accuracy: 0.0000e+00
Epoch 2/20
6/6 [==============================] - 0s 11ms/step - loss: 0.6754 - accuracy: 0.0015 - val_loss: 0.5230 - val_accuracy: 0.0000e+00
Epoch 3/20
6/6 [==============================] - 0s 11ms/step - loss: 0.4636 - accuracy: 0.0052 - val_loss: 0.3621 - val_accuracy: 0.0180
Epoch 4/20
6/6 [==============================] - 0s 11ms/step - loss: 0.3337 - accuracy: 0.0180 - val_loss: 0.2831 - val_accuracy: 0.0240
Epoch 5/20
6/6 [==============================] - 0s 12ms/step - loss: 0.2704 - accuracy: 0.0240 - val_loss: 0.2408 - val_accuracy: 0.0479
Epoch 6/20
6/6 [==============================] - 0s 10ms/step - loss: 0.2299 - accuracy: 0.0449 - val_loss: 0.2096 - val_accuracy: 0.0719
Epoch 7/20
6/6 [==============================] - 0s 10ms/step - loss: 0.1983 - accuracy: 0.0502 - val_loss: 0.1827 - val_accuracy: 0.0449
Epoch 8/20
6/6 [=

In [10]:
x_train_normal_pred  = autoencoder.predict(x_train_normal)
mse = np.mean(np.square(x_train_normal_pred-x_train_normal),axis=1)
threshold = np.percentile(mse,95)

x_test_pred = autoencoder.predict(x_test)
mse = np.mean(np.square(x_test_pred-x_test),axis=1)
y_pred = (mse > threshold).astype(int)

32/32 [==============================] - 0s 1ms/step


In [11]:
from sklearn.metrics import confusion_matrix,classification_report

print(confusion_matrix(y_test,y_pred))

[[379  31]
 [126 464]]


In [12]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.75      0.92      0.83       410
           1       0.94      0.79      0.86       590

    accuracy                           0.84      1000
   macro avg       0.84      0.86      0.84      1000
weighted avg       0.86      0.84      0.84      1000

